In [1]:
import os
if 'PoC' not in os.listdir(os.curdir):
    %cd ../
import scraper.lemonde_scraper
import numpy as np
from scipy.sparse import find
feats = scraper.lemonde_scraper.loadFeaturesArticlesAsJson( "data/features")

/home/eric/GitGalaxy/news-scraper


# Preprocessing

## Select the data

For each article gather all the text I want to search

In [2]:
news_body = []
for feat in feats:
    news_body.append( feat['title'] + '\n' + 
                      feat['article_description'] + '\n' + 
                      feat['article_content']
                    )

## Transform the text to features vector

The TfIdf has been selected as it is an easy and fast way to have a search engine.

In [23]:
f = open("ranking/fr_stop_words.txt")
words = f.read()
stop_words = words.split('\n')

for i in range(len(stop_words)-1, -1, -1):
    if stop_words[i] == "" or stop_words[i][0] == "#":
        del stop_words[i]

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

text_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)),
                     ('tfidf', TfidfTransformer())])

In [25]:
news_body_tfidf = text_clf.fit_transform( news_body )
news_body_tfidf.shape

(116, 13775)

# About the data

In [11]:
import seaborn

In [18]:
len(stop_words)

496

# Ranking

In [26]:
dd = text_clf.steps[0][1].get_feature_names()

In [34]:
query = "le gouvernement facebook"
# TODO: lowercase query, remove 0 score
cumul_tfidf = np.zeros((116,1))

words_to_query = []
for w in query.split():
    if w in dd:
        words_to_query.append(dd.index(w))
    elif w in stop_words:
        print(w + " is a stop word.")
    else:
        print(w + " doesn't appear in any article.")
        
for w_index in words_to_query:
    article_index, _, tfidf_query = find(news_body_tfidf[:, w_index])
    for index, a_index in enumerate(article_index):
        cumul_tfidf[a_index] = cumul_tfidf[a_index] + tfidf_query[index]
        
sort_articles = cumul_tfidf.argsort(axis=0)[::-1].flatten() # Descending sorting
for i in sort_articles[0:10]:
    print(" * " + feats[i]['title'])
    print(" ----> Score: " + str(cumul_tfidf[i]) )

le is a stop word.
 * L’intégrité de Facebook « plus importante que les profits », selon Mark Zuckerberg
 ----> Score: [ 0.35950176]
 * Entrée à l’université : la sélection toujours au cœur du débat
 ----> Score: [ 0.16659863]
 * La numéro deux du gouvernement espagnol présentée à tort comme la fille d’un général franquiste
 ----> Score: [ 0.15708136]
 * Catalogne : Puigdemont, sous le coup d’un mandat d’arrêt européen, disposé à être candidat
 ----> Score: [ 0.09479473]
 * La bataille continue sur la réserve parlementaire
 ----> Score: [ 0.08933527]
 * L’Assemblée vote des coupes claires dans les aides au logement
 ----> Score: [ 0.08215112]
 * Richard Ford : « Nous, Américains, avons perdu de vue la vérité »
 ----> Score: [ 0.0810098]
 * Fin de la sécu étudiante : colère et inquiétude des mutuelles
 ----> Score: [ 0.07525073]
 * Attaque à New York, Catalogne, harcèlement : les actualités à retenir cette semaine
 ----> Score: [ 0.06875084]
 * Le bonus-malus : « Une juste responsabilis

In [28]:
"le" in dd

False